# Local Plan Reference Column Reassignment

## Overview
This notebook reassigns reference columns for local plan data to improve consistency and clarity across datasets.

## Purpose
The primary goals are to:
1. **Create standardised references** for local plans following a consistent naming format
2. **Handle joint local plans** differently by using the joint plan name instead of individual authority names
3. **Apply these new references** across related datasets (local plan documents and timelines)
4. **Resolve data quality issues** where document references don't match their source local plan references

## Workflow

### 1. Reference Creation (`reference_func`)
Generates new reference columns (`reference2`) for the main local plan dataset (`lp`) with the format:
- **Single local plans**: `{organisation-slug}-local-plan-{year}`
  - Example: `epping-forest-district-council-local-plan-2011`
- **Joint local plans**: `{joint-plan-name-slug}` (no year suffix)
  - Example: `the-babergh-and-mid-suffolk-joint-local-plan`

Joint plans are identified by the presence of a semicolon (`;`) in the `organisations` column.

### 2. Lookup Generation
Creates mapping dictionaries from old references to new references:
- `ref_lookup`: Maps old local plan references to new ones

### 3. Application to Related Datasets
Applies the new references to the `lp_doc` (local plan documents) dataset by:
- Matching documents to their source local plan using the `local-plan` column
- Looking up the new reference from the mapping
- Handling duplicates by adding unique suffixes (`-1`, `-2`, `-3`, etc.)

### 4. Data Quality Resolution
Identifies and fixes mismatches where document `local-plan` values don't exist in the source data, ensuring all references resolve correctly.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Read in source data to be changed
lp = pd.read_csv('../export/archive/local-plan.csv')
lp_doc = pd.read_csv('../export/archive/local-plan-document.csv')
lp_time = pd.read_csv('../export/archive/local-plan-timetable.csv')
lp_bound = pd.read_csv('../export/archive/local-plan-boundary.csv')

# Read in LPA name lookup
lpa_lookup = pd.read_csv('supplementary-data/lpa-lookup.csv')
org_lookup = pd.read_csv('supplementary-data/organisation.csv')

## Local plans

In [3]:
# Map organisation names
lp['organisation-name'] = lp['organisations'].map(
    lpa_lookup.set_index('organisation')['organisation_label']
)

# Replace missing Lincolnshire County Council name
lp.loc[lp['organisations'] == 'local-authority:LIN', 'organisation-name'] = 'Lincolnshire County Council'

In [4]:
def reference_func(source_df, lpa_col_name):
    """
    Create a reference column for each row in the dataframe.
    Format: {organisation-slug}-local-plan-{year}
    
    For joint local plans (organisations column contains ';'), uses slugified 'name' column only.
    Otherwise uses {lpa-slug}-local-plan-{year}.
    
    Uses period-start-date for year, falls back to period-end-date if NaN,
    then falls back to empty string if both are NaN.
    
    Args:
        source_df: DataFrame with 'organisation-name', 'period-start-date', 
                   and 'period-end-date' columns
    
    Returns:
        DataFrame with new 'reference' column
    """
    df = source_df.copy()
    
    # Check if it's a joint local plan (organisations contains ';')
    is_joint = df['organisations'].str.contains(';', na=False)
    
    # Create slug for non-joint (from lpa_col_name)
    regular_slug = (
        df[lpa_col_name]
        .str.replace(' ', '-')
        .str.lower()
    )
    
    # Create slug for joint (from 'name')
    joint_slug = (
        df['name']
        .str.replace(' ', '-')
        .str.lower()
    )
    
    # Extract year: try period-start-date first, fallback to period-end-date, then ''
    year = df['period-start-date'].fillna(df['period-end-date']).fillna('').astype(str)
    year = year.str.replace('.0', '')
    
    # Generate reference
    # Joint plans: just the name slug
    # Non-joint: {slug}-local-plan-{year}
    regular_reference = regular_slug + '-local-plan-' + year
    regular_reference = regular_reference.str.rstrip('-')
    
    df['reference2'] = np.where(is_joint, joint_slug, regular_reference)
    
    return df

In [5]:
# Create new reference column in local-plans
lp = reference_func(lp, lpa_col_name='organisation-name')

In [6]:
# Create a lookup of old to new references
ref_lookup = lp.set_index('reference')['reference2'].to_dict()

In [7]:
# Save updated local-plans to CSV in both fixed-data and export folders
lp = lp.drop(columns=['reference']).rename(columns={'reference2': 'reference'})

## Documents

In [8]:
# Update the Cambridge row to use the correct local-plan reference
lp_doc.loc[lp_doc['reference'] == 'cambridge-local-plan', 'local-plan'] = 'cambridge-city-council-local-plan-2018'

# Map the local-plan references in lp_doc to the new reference2 values
lp_doc['reference2'] = lp_doc['local-plan'].map(ref_lookup)
lp_doc['local-plan'] = lp_doc['local-plan'].map(ref_lookup)

# Add numbers to the end of the reference2 columns
# Identify which reference2 values are duplicated
counts = lp_doc['reference2'].value_counts()
duplicated_refs = counts[counts > 1].index

# Get cumulative count within each group (starting from 1)
counter = (lp_doc.groupby('reference2').cumcount() + 1).fillna(0).astype(int)

# Add suffix only to duplicates
mask = lp_doc['reference2'].isin(duplicated_refs)
lp_doc['reference2'] = lp_doc['reference2'].where(~mask, lp_doc['reference2'] + '-' + counter.astype(str))

In [9]:
# Find which local-plan values don't have a mapping
missing_mappings = lp_doc[lp_doc['reference2'].isna()]['local-plan'].unique()
print("Local plans with no mapping:")
print(missing_mappings)

# Check if they exist in the lp dataframe
for lp_ref in missing_mappings:
    exists_in_lp = lp[lp['reference'] == lp_ref].shape[0] > 0
    print(f"  {lp_ref}: {exists_in_lp}")

Local plans with no mapping:
[]


In [10]:
# Save updated local-plan-document to CSV
lp_doc.drop(columns=['reference']).rename(columns={'reference2': 'reference'}).to_csv('../export/local-plan-document.csv', index=False)

## Timetables

In [11]:
# Map the local-plan references in lp_doc to the new reference2 values
lp_time['reference2'] = lp_time['local-plan'].map(ref_lookup) + '-' + lp_time['local-plan-event']
lp_time['local-plan'] = lp_time['local-plan'].map(ref_lookup)

# Add numbers to the end of the reference2 columns
# Identify which reference2 values are duplicated
counts = lp_time['reference2'].value_counts()
duplicated_refs = counts[counts > 1].index

# Get cumulative count within each group (starting from 1)
counter = (lp_time.groupby('reference2').cumcount() + 1).fillna(0).astype(int)

# Add suffix only to duplicates
mask = lp_time['reference2'].isin(duplicated_refs)
lp_time['reference2'] = lp_time['reference2'].where(~mask, lp_time['reference2'] + '-' + counter.astype(str))

In [12]:
# Check if there are any more duplicates (returns either True or False)
lp_time['reference2'].duplicated().any()

np.False_

In [13]:
# Save updated local-plan-document to CSV
lp_time.drop(columns=['reference']).rename(columns={'reference2': 'reference'}).to_csv('../export/local-plan-timetable.csv', index=False)

## Boundaries

In [14]:
lp_bound

,entry-date,start-date,end-date,reference,name,organisations,geometry
0,2024-10-14,NaN,NaN,E09000014,London Borough of Haringey,local-authority:HRY,"MULTIPOLYGON (((-0.127877 51.609441, -0.128667..."
1,2024-10-14,NaN,NaN,E07000106,Canterbury City Council,local-authority:CAT,"MULTIPOLYGON (((1.122791 51.296104, 1.122001 5..."
2,2024-10-14,NaN,NaN,E07000009,East Cambridgeshire District Council,local-authority:ECA,"MULTIPOLYGON (((0.239515 52.508261, 0.238788 5..."
3,2024-10-14,NaN,NaN,E07000009,East Cambridgeshire District Council,local-authority:ECA,"MULTIPOLYGON (((0.239515 52.508261, 0.238788 5..."
4,2024-10-14,NaN,NaN,E09000029,London Borough of Sutton,local-authority:STN,"MULTIPOLYGON (((-0.17138 51.392479, -0.172463 ..."
...,...,...,...,...,...,...,...
76,2024-10-14,NaN,NaN,E09000029,London Borough of Sutton,local-authority:STN,"MULTIPOLYGON (((-0.17138 51.392479, -0.172463 ..."
77,2024-10-14,NaN,NaN,E08000009,Trafford Metropolitan Borough Council,local-authority:TRF,"MULTIPOLYGON (((-2.281586 53.464866, -2.281903..."
78,2024-10-14,NaN,NaN,E07000082,Stroud District Council,local-authority:STO,"MULTIPOLYGON (((-2.320848 51.847226, -2.321146..."
79,2024-10-14,NaN,NaN,E07000241,Welwyn Hatfield Borough Council,local-authority:WEW,"MULTIPOLYGON (((-0.183552 51.860116, -0.183575..."


In [15]:
# Load the mapping from organisation.csv
lad_to_lpa = dict(zip(org_lookup['local-authority-district'], org_lookup['local-planning-authority']))

# Map LAD codes to LPA codes
lp_bound['reference'] = lp_bound['reference'].map(lad_to_lpa)

# Save the updated CSV
lp_bound.to_csv('../export/local-plan-boundary.csv', index=False)

In [ ]:
# Fix the local plan boundaries in the original local plan CSV too
lp['local-plan-boundary2'] = lp['local-plan-boundary'].map(lad_to_lpa)

# Do manual lookups for remaining missing boundaries
## NOTE -- Lincolnshire County Council is not included here as it does not have a LPA boundary
manual_lookups = {
    'Local Plan for the Broads': 'E60000326',
    'Broads Local Plan Review': 'E60000326',
    'Local Plan Review 2018-2036': 'E60000318',
    'New Local Plan': 'E60000278',
    'New Ashford Local Plan to 2042': 'E60000253'}

lp['local-plan-boundary2'] = np.where(lp['local-plan-boundary2'].isnull(),
         lp['name'].map(manual_lookups),
         lp['local-plan-boundary2']
            )

In [17]:
lp.loc[lp['local-plan-boundary2'].isnull()]

,entry-date,start-date,end-date,name,organisations,description,period-start-date,period-end-date,local-plan-boundary,documentation-url,adopted-date,organisation-name,reference,local-plan-boundary2
137,2024-10-02,NaN,NaN,Lincolnshire County Council Local Plan 2023,local-authority:LIN,NaN,2023.0,2040.0,NaN,https://www.n-kesteven.gov.uk/central-lincolns...,2023-04-13,Lincolnshire County Council,lincolnshire-county-council-local-plan-2023,NaN


In [18]:
lp = lp.drop(columns=['local-plan-boundary']).rename(columns={'local-plan-boundary2': 'local-plan-boundary'}).to_csv('../export/local-plan.csv', index=False)